In [42]:
import json
import numpy as np
from itertools import groupby
from sklearn.cluster import Birch, AffinityPropagation, MiniBatchKMeans, MeanShift, estimate_bandwidth, KMeans, DBSCAN
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/10k.test.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 2717122


In [5]:
stopwords = set(one_time)

In [6]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Март 2015

In [7]:
mart_news = news[:500]
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 500


In [8]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)

tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 15113


# Birch

In [15]:
km = Birch(n_clusters=50).fit(tfidf_matrix)
labels = km.labels_

In [17]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 34, titles:
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Киевляне установили возле российского посольства деревянные кресты
	Украинские военные требуют отставки Порошенко
	Москаль рассказал о попытках воздействовать на силовиков российской попсой
	Пока силовики обстреливают Донецк, Порошенко призывает к миру
	"Яценюк, выходи!": премьер сыграл с шахтерами в прятки
	«Начать жизнь с нуля»
	На въездах в Киев выставят снайперов
	Под Дебальцево убит «генерал Ичкерии»
	"Дело Carlton": борьба за нравственность или большая политика?
	ООН: количество погибших на Украине превысило 5350 человек
	Кадырова признали главным борцом с пьянством в России
	Штурм администрации Порошенко: первая попытка отражена
	В Киеве митингующие штурмуют здание администрации Порошенко
	Рамзан Кадыров - человек года в борьбе с наркотиками и алкоголизмом в России
	Украина не может счастливой стать - вопреки законам Природы
	США признали отсутствие полного контроля России на

# AffinityPropagation

In [20]:
km = AffinityPropagation().fit(tfidf_matrix)
labels = km.labels_

In [21]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 5, titles:
	Гроза разбудила москвичей утром 1 февраля
	Погода преподнесла сюрприз жителям столичного региона
	Снежную бурю прочувствует треть населения США
	А снег идет: синоптики объявили оранжевый уровень опасности
	Дыхание Атлантики: Россию ожидают снежные выходные
Cluster: 1, count news: 4, titles:
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Главы ДНР и ЛНР: мы готовы остановиться там, где находимся сейчас
Cluster: 2, count news: 3, titles:
	«Резкое повышение невозможно»
	Пять истин, которые помогут вам выжить в кризис
	Мэр Киева объяснил, что подорожавший проезд в метро на самом деле подешевел
Cluster: 3, count news: 4, titles:
	Спасенные от огня книги библиотеки ИНИОН теперь спасают от воды
	ИНИОН — что мы потеряли?
	Пожар в ИНИОНе: специалисты войдут внутрь и оценят масштабы ущерба
	Залитые водой при тушении 

# KMeans

In [43]:
km = KMeans(n_clusters=50, n_jobs=-1).fit(tfidf_matrix)
labels = km.labels_

In [44]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 8, titles:
	Шувалов: правительство не будет "проламывать" болезненные реформы
	Шувалов: стабильного рубля не будет
	Банк России увидел предпосылки для укрепления рубля
	Эльвира Набиуллина: Рубль теперь скорее укрепится, чем упадет
	Турция будет доплачивать за отдых российских туристов
	Жители Владивостока предложили ЦБ напечатать купюры в 2 тысячи рублей
	Болгария будет давать российским туристам визы на 3 и 5 лет
	Где москвичу с горы скатиться
Cluster: 1, count news: 7, titles:
	Игровые трейлеры недели: загробный туризм и ярость ниндзя
	«Начать жизнь с нуля»
	На въездах в Киев выставят снайперов
	ООН: количество погибших на Украине превысило 5350 человек
	Киев отказался от Дня Победы и Великой Отечественной
	Депутаты хотят признать передачу Крыма Украине в 1954 году незаконной
	Момент истины: как остановить войну на Украине
Cluster: 2, count news: 21, titles:
	Пока силовики обстреливают Донецк, Порошенко призывает к миру
	Перед встречей с Путиным Олланд и Мерке

# MiniBatchKMeans

In [30]:
km = MiniBatchKMeans(n_clusters=50, max_iter=300).fit(tfidf_matrix)
labels = km.labels_

In [40]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 500, titles:
	Боевики "Исламского государства" обезглавили второго японского заложника
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Исламисты опубликовали видео с казнью второго японского заложника
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	Гроза разбудила москвичей утром 1 февраля
	Грузовики МЧС, доставившие продукты и медикаменты на юго-восток Украины, вернулись в Россию
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Видео убийства Кэндзи Гото: боевики ИГ пообещали, что устроят для Японии кошмар
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Ополченцы сообщают, что полностью контролируют Дебальцевский котел
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомобил

# DBScan

In [50]:
db = DBSCAN(eps=1.2, min_samples=3).fit(tfidf_matrix)
labels = db.labels_

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 24
-1: 246, 0: 12


In [51]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 246, titles:
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Гроза разбудила москвичей утром 1 февраля
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомобиля
	Погода преподнесла сюрприз жителям столичного региона
	Командира батальона "Донбасс" контузило
	«Резкое повышение невозможно»
	Бойцы "Азова" потеряли четыре танка
	Новосибирский спортсмен с протезом ноги покорил алтайскую гору и стал Альпинистом России
	Игровые трейлеры недели: загробный туризм и ярость ниндзя
	Киевляне установили возле российского посольства деревянные кресты
	Украинские военные требуют отставки Порошенко
	Скидок больше не будет
	Патриарх Кирилл: Церковь не должна жить в своем гетто
	На этапе Кубка мира Вик Уайлд занял первое место в параллельном 

# MeanShift

In [39]:
X = tfidf_matrix.toarray()
bandwidth = estimate_bandwidth(X, quantile=0.2)
km = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(X)
labels = km.labels_

In [41]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 500, titles:
	Боевики "Исламского государства" обезглавили второго японского заложника
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Исламисты опубликовали видео с казнью второго японского заложника
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	Гроза разбудила москвичей утром 1 февраля
	Грузовики МЧС, доставившие продукты и медикаменты на юго-восток Украины, вернулись в Россию
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Видео убийства Кэндзи Гото: боевики ИГ пообещали, что устроят для Японии кошмар
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Ополченцы сообщают, что полностью контролируют Дебальцевский котел
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомобил

## Сохранение данных

In [48]:
news_clusters = {}
for i in range(0, len(labels)):
    n = mart_news[i]
    l = labels[i]
    news_clusters[n.id] = int(l)

In [49]:
cluster_file = '/data/kasandra/year/result/50_kmens.json'

with open(cluster_file, encoding="utf8", mode="w") as f:
    d_json = json.dumps(news_clusters, ensure_ascii=False)
    f.write(d_json + '\n')